In [2]:
%load_ext autoreload
%autoreload 2

In [7]:
import interp.cui as cui
import jax.numpy as jnp
import jax

await cui.init(port=6789)  # type: ignore

started
Composable UI initialized! Make sure you've run `npm install` and `npm run start` in /interp/app before using


Exception in thread Thread-6:
Traceback (most recent call last):
  File "/home/ubuntu/conda/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/conda/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/unity/interp/cui.py", line 52, in loop_in_thread
    loop.run_until_complete(websockets.serve(handler, "localhost", port))
  File "/home/ubuntu/conda/lib/python3.9/asyncio/base_events.py", line 642, in run_until_complete
    return future.result()
  File "/home/ubuntu/conda/lib/python3.9/asyncio/tasks.py", line 690, in _wrap_awaitable
    return (yield from awaitable.__await__())
  File "/home/ubuntu/conda/lib/python3.9/site-packages/websockets/legacy/server.py", line 1086, in __await_impl__
    server = await self._create_server()
  File "/home/ubuntu/conda/lib/python3.9/asyncio/base_events.py", line 1494, in create_server
    raise OSError(err.errno, 'error while attempting '
OSError

In [4]:
from interp.ui.very_named_tensor import VeryNamedTensor

from interp.model.model_loading import load_model
from interp.tools.log import LogCacheAll, LogCache
from interp.tools.interpretability_tools import get_activations_all_layers

model, params, tok = load_model("GPT2",models_dir="/home/ubuntu/interpretability_models_jax", dtype=jnp.float16)

INFO:absl:Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker: 


loading GPT2 from /home/ubuntu/interpretability_models_jax/GPT2


INFO:absl:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.


In [5]:
text1 = "Yesterday Sarah walked around the park with their dog"
text2 = "Yesterday John walked around the park with their dog"
# text1 = "On Saturday, Mr Smith went to the mall."
# text2 = "On Saturday, Mrs Smith went to the mall."
token_ids_1 = tok(text1, padding=False, return_tensors="jax")["input_ids"]
token_ids_2 = tok(text2, padding=False, return_tensors="jax")["input_ids"]

acts1 = get_activations_all_layers(model,params,token_ids_1,["attention.inp","mlp.inp","mlp.out","attention.out_by_head"])
acts2 = get_activations_all_layers(model,params,token_ids_2,["attention.inp","mlp.inp","mlp.out","attention.out_by_head"])



token_strs = [tok.decode(token_id) for token_id in token_ids_1[0]]

# Let's visualize the attention scores by layer and head
# does pytree stuff work?
print({k:v.shape for k,v in acts1.items()})

attn_inp_diffs = (acts1["attention.inp"]-acts2["attention.inp"]).squeeze(1)

vnt_attn = VeryNamedTensor(
    attn_inp_diffs,
    dim_names="layer seq hidden".split(),
    dim_types="layer seq hidden".split(),
    dim_idx_names=[
        [str(i) for i in range(model.num_layers)],
        token_strs,
        [str(i) for i in range(model.hidden_size)],
    ],
    units="prob",
    title="Layer Inp Diffs",
)

attn_outp_diffs = (acts1["attention.out_by_head"]-acts2["attention.out_by_head"]).squeeze(1)

vnt_attn_out = VeryNamedTensor(
    attn_outp_diffs,
    dim_names="layer head seq hidden".split(),
    dim_types="layer head seq hidden".split(),
    dim_idx_names=[
        [str(i) for i in range(model.num_layers)],
        [str(i) for i in range(model.num_heads)],
        token_strs,
        [str(i) for i in range(model.hidden_size)],
    ],
    units="prob",
    title="Head Outp Diffs",
)

mlp_outp_diffs = (acts1["mlp.out"]-acts2["mlp.out"]).squeeze(1)

vnt_mlp = VeryNamedTensor(
    mlp_outp_diffs,
    dim_names="layer seq hidden".split(),
    dim_types="layer seq hidden".split(),
    dim_idx_names=[
        [str(i) for i in range(model.num_layers)],
        token_strs,
        [str(i) for i in range(model.hidden_size)],
    ],
    units="prob",
    title="Mlp Outp Diffs",
)

{'logits': (1, 9, 50257), 'attention.inp': (12, 1, 9, 768), 'mlp.inp': (12, 1, 9, 768), 'mlp.out': (12, 1, 9, 768), 'attention.out_by_head': (12, 1, 12, 9, 768)}


In [6]:
# Run this and then click the link that appears.
await cui.show_tensors(
    vnt_attn,
    vnt_mlp,
    vnt_attn_out,
)

In [5]:

# %%

handler
init
nameStartup
name untitled
handler
init
nameStartup
name untitled
callback


In [6]:
dict(log1)

KeyError: 0